# Overwatch: Predicting Competitive Rank

## Background

Overwatch Origins Edition was released on May of 2016 for Windows, PS4 and Xbox One, by Blizzard Entertainment. Blizzard is known for their Massive Multiplayer Online (MMO) and stragety based games, like World of Warcraft, Heroes of the Storm, and StraCraft. Overwatch features a team based first-person shooter with 26 characters split between four group classifications; Offense, Defense, Tanks, and Support. The structure of the game is based on team work to capture objectives and
transport cargo objectives, or hold capture points. The games design has a variety of modes to choose from, depending on the players preferred gaming style; Arcade mode, which presents challenges for a player to play solo or as a team, Quickplay mode, where players are queued with other players in non-ranked matches for the maps objectives, and Competitive mode, where players are queued with other players based on skill rating for map objectives and is only available during seasons.

Since it's release, the game has grown in popularity, with the most recent player count at 35 million in October of 2017, according to statista.com (2017) and pcgamesn.com (2017). Overwatch has entered into the esports community with the launch of Overwatch League (overwatchleague.com/en-us/) in 2018, showcasing the top competitive players in the Overwatch community. The competitive players are determined through the games Skill Rating system in Competitive matches, ranging from 1 to 4000+ (overwatch.gamepedia.com/Competitive_Play), with Bronze being the lowest and Grand Master being the highest (outside of the top 500 for the region).

With the demomstrated interest from the gaming community on Overwatch League, at 10 million viewers in the League's first week (dotesports.com/overwatch/news/overwatch-league-10-million-viewers-20274), Competitive Overwatch is a popular game mode. As a regular gamer, and an Overwatch fan, I regularly participate in Competitive matches, and I am working to rise in the ranks with each passing season. During my time playing Overwatch Competitive, I have tried to maintain a balance in the number of characters I play well, with the hope this will help me raise my Skill Rating. The Skill Rating system has been a topic in numerous gaming threads on reddit and Overwatch forums, as the community tries to determine the best way to rise in the ranks. My goal is to determine what is the best way to increase a player's skill rating, thus their Competitive Rank, by analyzing player's Overwatch console data.

---
## Problem Statement

**Does having the flexibility to play 2-3 characters increase your chances of a higher Skill Rating (SR), or should a player focus on 1 character and master them for a higher SR? Can I predict a player's SR from the number of characters they have played, by analyzing the total number of games played and the total wins/loss/ties for each character, from a span of three seasons? **

---
### Hypothesis

I believe that having the flexibilty to play 2-3 characters, well, provides a player the ability to adapt to a competitive match's team composition by filling in the needed group classification for that match. By creating a balanced team, the chances of winning the match increases, thus providing the player with a higher SR through out their competitive season.

By analyzing a player's game data, I will be able to predict their future seasons SR based on the number of characters they've played and the total games played with each character over a season. 


---
## Data
The data for this project was obtained by contacting Blizzard Entertainment through Omnicmeta.com. The data was provided for Season 5, 6, and 7 for about 100k players on combined Xbox/PS4 console. The data columns contain the player id (numerically randomized), SR, and wins/losses/ties for each hero (26 x 3).

The data is classified as confidential by the Blizzard and therefore can not be made public. Using a python script, I have created a dummy data set, with similar data to use as a representation of the data prepping work I will be performing on the data obtained from Omnicmeta. The final results will reflect the data from Omnicmeta.

---
### Data Import

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt

# Create path for csv file
url = 'C:\Users\Chupi\Documents\GADataScience\Project_Work\OverwatchRankingsProject\owdata_dummy.csv'
#creating dataframe for the dummy data
OW = pd.read_csv(url, skipinitialspace = True)
OW.head()

,player_id,skillrating,Genji_wins,Genji_losses,Genji_ties,McCree_wins,McCree_losses,McCree_ties,Pharah_wins,Pharah_losses,...,Zenyatta_ties,Orisa_wins,Orisa_losses,Orisa_ties,Doomfist_wins,Doomfist_losses,Doomfist_ties,Moira_wins,Moira_losses,Moira_ties
0,2,2258,0,0,0,0,0,11,0,0,...,0,0,0,0,0,0,0,2,0,0
1,3,808,0,0,0,5,0,0,0,2,...,6,0,0,0,4,0,0,0,0,0
2,4,3408,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,5,2021,0,103,87,139,0,0,0,0,...,75,123,0,0,0,0,0,0,0,0
4,6,1562,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,3,0,0,0


---
### Processing the data
Because the data is so large, we need to reduce the amount of information we have by creating new columns for the total number of games played in the season, total number of games played by each character divided by the total number of games played, and then, the total characters played for each season. The ultimate goal is to have a dataframe with 5 columns as follows:

* Player
* SR
* Rank status
* Total Games Played
* Total Characters played

This will require several steps. First step is to remove the whitespaces in the file, as they were disrupting the syntax for furture operations. 

Then we want to find the total number of games played by each player. This was acheived by creating a list of columns with each character. This column was then added to the end of the dataframe

From here, we will take the character column and divide by the total number of games played to create a new column of the percentage played of each character for the season (character ratio). After this, we will want to determine the most played characters, based on the the character ratio.

Next will be to count the total number of characters, whose sum will equal 80% of the total games played. To ensure I am including characters with higher game time played, I am setting a parameter that will exclude characters from the sum, whose cell values are less than or equal to 5%.

In order to create the Logistic Model, I created a new column that categorizes the SR column into their corresponding Rank per https://overwatch.gamepedia.com/Competitive_Play (20 January 2018).

Bronze - 1-1499 SR

Silver - 1500-1999 SR

Gold - 2000-2499 SR

Platinum - 2500-2999 SR

Diamond - 3000-3499 SR

Master - 3500-3999 SR

Grandmaster - 4000+

---

In [2]:
#Adding a new column totalGames that is the sum of the character columns
col_list = list(OW.columns.values)

col_list = col_list[2:79]
OW['totalGames'] = OW[col_list].sum(axis = 1)

#------------------------------------------------------------------#

OW.head()
#Create a blank df
OW_new = pd.DataFrame()

#extracting the character name by splitting the column name by '_' to yield the first element
for name in col_list:
    char_name = name.split('_')[0]
    column = OW[name]
#Check if this column exists in the dataframe. If it does not, then add it.
    if not char_name in OW_new.columns:
        OW_new[char_name] = column
    #IF it already exist, sum it.
    else:
        prev_col = OW_new[char_name]
        OW_new[char_name] = column + prev_col
        
#------------------------------------------------------------------#

#Create a blank df
OW_ratio = pd.DataFrame()

#redefine col_list
col_list = list(OW_new.columns.values)

#Add the columns player, SR and totalGames to the OW_ratio df
OW_ratio['player'] = OW_new['player']
OW_ratio['SR'] = OW_new['skillrating']
OW_ratio['totGames'] = OW_new['totalGames']

#Finding the charater play time ratio (total character games divided by total games) starting at index 2
for name in col_list[2:28]:
    char_total = OW_new[name]
    total_games = OW_new['totalGames']
    OW_ratio[name] = char_total.div(total_games, axis = 0)

#------------------------------------------------------------------#

# Create a new empty df
OW = pd.DataFrame()

#redefine col_list to OW_ratio df
col_list = list(OW_ratio.columns.values)

# Pulling columns to new df
OW['player'] = OW_ratio['player']
OW['SR'] = OW_ratio['SR']
OW['totGames'] = OW_ratio['totGames']

# Count the number of columns, whose sum is at least 80%, but does not include values under 5%.
min_value = 0.05
max_tot = 0.80
#creating an empty list for the results to populate later
col_counter = []
for index, row in OW_ratio.iterrows():
    counter = 0
    sum = 0
    for cell in row[3:29]:
        if cell > min_value:
            counter = counter + 1
            sum = sum + cell
            if sum >= max_tot:
                break;
    # filling in the empty list with the results of counter
    col_counter.append(counter)
# creating a new column that is set to the total count
OW['tot_char'] = col_counter

#------------------------------------------------------------------#

# Adding a new column for categorizing the SR into a string Rank. First create an empty list for the results of the for loop

col_rank = []
for value in OW['SR']:
    if value <= 1499:
        rank = 'Bronze'
    elif value <= 1999:
        rank = 'Silver'
    elif value <= 2499:
        rank = 'Gold'
    elif value <= 2999:
        rank = 'Platinum'
    elif value <= 3499:
        rank = 'Diamond'
    elif value <=3999:
        rank = 'Master'
    elif value > 4000:
        rank = 'Grandmaster'
    # filling in the empty list with the results of rank
    col_rank.append(rank)
# adding rank column to the df
OW['rank'] = col_rank

OW.head()

KeyError: 'player'

---
## Data Analysis: Predicting Rank

After the processing the data on season 5, 6, and 7, the three dataframes were merged together to create 1 master table.  

``` 
OW = [owS5, owS6, owS7]  
OW_master = pd.concat(OW)   
```

The master table was exported to csv so it the final data could be used for the modeling on this project.

```
OW_master.to_csv('C:\Users\Chupi\Documents\GADataScience\Project_Work\OverwatchRankingsProject\OW_master.csv', index = False)
```

We will now use logistic regression to predict Overwatch rank on the master dataframe.

---

In [ ]:
#Reading the final Overwatch data into pandas
url = 'C:\Users\Chupi\Documents\GADataScience\Project_Work\OverwatchRankingsProject\OW_master.csv'
OW_master = pd.read_csv(url)
OW_master.head(10)

In [ ]:
# gathering statistics of the data
feature_col = ['SR','totGames', 'tot_char']
OW_master[feature_col].describe()

In [ ]:
# Evaluating the correlation of the features within the data frame
OW_master[feature_col].corr()

In [ ]:
# evaluating the data in box plot of SR and tot_char
OW_master['SR'].plot(kind = 'box')
OW_master['tot_char'].plot(kind = 'box')

In [ ]:
# evaluating the data in a scatter plot
sns.lmplot(x = 'tot_char', y = 'SR', data = OW_master, ci = None,line_kws={'color': 'red'})
sns.lmplot(x = 'totGames', y = 'SR', data = OW_master, ci = None, line_kws={'color': 'red'})

In [ ]:
# setting up Logistic Regression Model
logreg = LogisticRegression()

# Creating an X feature and a y-response
f_cols = ['totGames', 'tot_char']
X = OW_master[f_cols]
y = OW_master.SR

# fitting the model
logreg.fit(X, y)

#create a column to store prediction
OW_Master['pred_rank'] = logreg.predict(X)

In [ ]:
# plot the class predictions
plt.scatter(OW_master.SR, OW_master.tot_char)
plt.plot(OW_master.SR, OW_master.pred_rank, color='red')
plt.xlabel('tot_char')
plt.ylabel('SR')

In [ ]:
# K means clustering evaluation

---
### Cited Sources: 
Overwatch player statistics: https://www.pcgamesn.com/overwatch/overwatch-sales-numbers, https://www.statista.com/statistics/618035/number-gamers-overwatch-worldwide/
